In [1]:
!mkdir -p ~/.config/Ultralytics/
!wget http://mirror.coggle.club/yolo/Arial.ttf -O ~/.config/Ultralytics/Arial.ttf
!wget http://mirror.coggle.club/yolo/Arial.Unicode.ttf -O ~/.config/Ultralytics/Arial.Unicode.ttf
!wget http://mirror.coggle.club/yolo/yolov8n-v8.2.0.pt -O yolov8n.pt
# !wget http://mirror.coggle.club/yolo/yolov8s-v8.2.0.pt -O yolov8s.pt
!wget http://mirror.coggle.club/yolo/yolov8m-v8.2.0.pt -O yolov8m.pt

--2025-07-21 21:46:12--  http://mirror.coggle.club/yolo/Arial.ttf
正在解析主机 mirror.coggle.club (mirror.coggle.club)... 222.134.1.30, 222.134.1.29, 222.134.1.28
正在连接 mirror.coggle.club (mirror.coggle.club)|222.134.1.30|:80... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度： 773236 (755K) [font/ttf]
正在保存至: ‘/home/lyz/.config/Ultralytics/Arial.ttf’

/home/lyz/.config/U 100%[===================>] 755.11K  3.62MB/s    用时 0.2s    

2025-07-21 21:46:13 (3.62 MB/s) - 已保存 ‘/home/lyz/.config/Ultralytics/Arial.ttf’ [773236/773236])

--2025-07-21 21:46:13--  http://mirror.coggle.club/yolo/Arial.Unicode.ttf
正在解析主机 mirror.coggle.club (mirror.coggle.club)... 222.134.1.28, 222.134.1.30, 222.134.1.29
正在连接 mirror.coggle.club (mirror.coggle.club)|222.134.1.28|:80... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度： 23278008 (22M) [application/x-font-ttf]
正在保存至: ‘/home/lyz/.config/Ultralytics/Arial.Unicode.ttf’

/home/lyz/.config/U 100%[===================>]  22.20M  6.52MB/s    用时 3.5s    

2025-07-21 21:46:16 (6.34 MB/s) - 已保存 ‘

In [2]:
!\rm -rf train_yolo

In [3]:
import os
import glob
import shutil

import os
import xml.etree.ElementTree as ET
from glob import glob

def convert_voc_to_yolo(xml_path, output_dir, class_names):
    """
    Convert Pascal VOC XML annotation to YOLO format
    
    Args:
        xml_path (str): Path to XML file
        output_dir (str): Directory to save YOLO txt files
        class_names (list): List of class names in order
    """
    # Parse XML
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    # Get image dimensions
    size = root.find('size')
    img_width = int(size.find('width').text)
    img_height = int(size.find('height').text)
    
    # Create output file path
    image_name = os.path.splitext(os.path.basename(xml_path))[0]
    txt_path = os.path.join(output_dir, f"{image_name}.txt")
    
    with open(txt_path, 'w') as f:
        for obj in root.iter('object'):
            # Get class name
            cls_name = obj.find('name').text
            if cls_name not in class_names:
                continue  # skip classes not in our list
            cls_id = class_names.index(cls_name)
            
            # Get bounding box coordinates
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            
            # Convert to YOLO format (center x, center y, width, height) - normalized
            x_center = ((xmin + xmax) / 2) / img_width
            y_center = ((ymin + ymax) / 2) / img_height
            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height
            
            # Write to file
            f.write(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

def main():
    # Configuration
    img_dir = "train"
    xml_dir = "train/XML/"  # Directory containing XML files
    img_dir = "train"
    output_dir = "train_yolo/train/labels"  # Directory to save YOLO format labels
    img_output_dir = "train_yolo/train/images"  # Directory to save YOLO format labels
    class_names = ["knife", "tool", "glassbottle", "drinkbottle", "metalbottle", "laptop", "umbrella", "pressuretank"]  # List your class names here, e.g., ["person", "car", "dog"]
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(img_output_dir, exist_ok=True)

    # Get all XML files
    xml_files = glob(os.path.join(xml_dir, "*.xml"))
    
    img_files = glob(os.path.join(img_dir, "*.jpg"))
    for path in img_files:
        shutil.copy(path, img_output_dir)
    
    # If class_names is empty, collect all unique class names from XML files
    if not class_names:
        unique_classes = set()
        for xml_file in xml_files:
            tree = ET.parse(xml_file)
            root = tree.getroot()
            for obj in root.iter('object'):
                cls_name = obj.find('name').text
                unique_classes.add(cls_name)
        class_names = sorted(list(unique_classes))
        print(f"Found classes: {class_names}")
    
    # Process each XML file
    for xml_file in xml_files:
        convert_voc_to_yolo(xml_file, output_dir, class_names)
    
    print(f"Conversion complete. YOLO format labels saved to {output_dir}")
    
main()

Conversion complete. YOLO format labels saved to train_yolo/train/labels


In [4]:
os.makedirs('train_yolo/val', exist_ok=True)
os.makedirs('train_yolo/val/labels', exist_ok=True)
os.makedirs('train_yolo/val/images', exist_ok=True)

labels = glob('train_yolo/train/labels/*')
images = glob('train_yolo/train/images/*')

labels.sort(); images.sort()

for x, y in zip(labels[-50:], images[-50:]):
    shutil.move(x, 'train_yolo/val/labels/')
    shutil.move(y, 'train_yolo/val/images/')

In [5]:
import os
dir_path = os.path.abspath('./') + '/'

# 需要按照你的修改path
with open('yolo.yaml', 'w', encoding='utf-8') as up:
    up.write(f'''
path: {dir_path}/
train: train_yolo/train/
val: train_yolo/val/

names:
    0: knife
    1: tool
    2: glassbottle
    3: drinkbottle
    4: metalbottle
    5: laptop
    6: umbrella
    7: pressuretank
''')

In [8]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import warnings
warnings.filterwarnings('ignore')


from ultralytics import YOLO
model = YOLO("yolov8m.pt")
results = model.train(data="./yolo.yaml", epochs=20, device=1, batch=8, imgsz=640)

New https://pypi.org/project/ultralytics/8.3.168 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.81 🚀 Python-3.11.8 torch-2.4.0+cu121 CUDA:1 (NVIDIA GeForce GTX 1080 Ti, 11163MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=./yolo.yaml, epochs=20, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=1, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fra

train: Scanning /home/lyz/Work/Kaggle/科大讯飞2025/X光安检图像识别挑战赛 赛季6/train_yolo/train/labels... 5528 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5528/5528 [00:06<00:00, 826.96it/s] 


train: New cache created: /home/lyz/Work/Kaggle/科大讯飞2025/X光安检图像识别挑战赛 赛季6/train_yolo/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /home/lyz/Work/Kaggle/科大讯飞2025/X光安检图像识别挑战赛 赛季6/train_yolo/val/labels... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<00:00, 1353.88it/s]

val: New cache created: /home/lyz/Work/Kaggle/科大讯飞2025/X光安检图像识别挑战赛 赛季6/train_yolo/val/labels.cache


...

      11/20      4.51G      1.027      1.129       1.22         19        640: 100%|██████████| 691/691 [04:29<00:00,  2.56it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  5.27it/s]

                   all         50        144      0.737      0.447      0.557      0.381



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      12/20      4.46G     0.9971      1.072      1.212         25        640:  61%|██████    | 421/691 [02:44<01:45,  2.55it/s]


KeyboardInterrupt: 

In [9]:
model = YOLO("runs/detect/train/weights/best.pt")

In [31]:
from glob import glob
import os
test_paths = glob('test1/*')
test_paths.sort()

all_result = []
for path in test_paths:
    predictions = model(path, save_txt=None)  # yolo model

    result = [[], [], [], [], [], [], [], []]
    for idx in range(len(predictions[0].boxes.cls)):
        # Get class name
        cls_id = int(predictions[0].boxes.cls[idx].item())
        x1, y1, x2, y2 = predictions[0].boxes.xyxy[idx].tolist()
        result[cls_id].append([x1, y1, x2, y2, predictions[0].boxes.conf[idx].item()])

    all_result.append(result)


image 1/1 /home/lyz/Work/Kaggle/科大讯飞2025/X光安检图像识别挑战赛 赛季6/test1/00000.jpg: 512x640 3 glassbottles, 17.6ms
Speed: 21.3ms preprocess, 17.6ms inference, 2.5ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 /home/lyz/Work/Kaggle/科大讯飞2025/X光安检图像识别挑战赛 赛季6/test1/00001.jpg: 640x640 7 glassbottles, 1 metalbottle, 15.8ms
Speed: 5.7ms preprocess, 15.8ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/lyz/Work/Kaggle/科大讯飞2025/X光安检图像识别挑战赛 赛季6/test1/00002.jpg: 640x544 1 glassbottle, 1 drinkbottle, 1 metalbottle, 14.3ms
Speed: 3.5ms preprocess, 14.3ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 544)

image 1/1 /home/lyz/Work/Kaggle/科大讯飞2025/X光安检图像识别挑战赛 赛季6/test1/00003.jpg: 608x640 2 glassbottles, 20.0ms
Speed: 6.6ms preprocess, 20.0ms inference, 2.5ms postprocess per image at shape (1, 3, 608, 640)

image 1/1 /home/lyz/Work/Kaggle/科大讯飞2025/X光安检图像识别挑战赛 赛季6/test1/00004.jpg: 416x640 4 glassbottles, 11.4ms
Speed: 1.7ms preprocess, 11.4ms infer

In [32]:
with open("submit.json", "w") as up:
    json.dump(all_result, up)

In [18]:
len(json.load(open("results_test1.json")))

1395